# Investopedia Scraper

## Imports

This installations and imports are needed to run Selenium etc. in Google Colab.

In [1]:
def initial_install():
#  !pip install selenium
#  !apt-get update # to update ubuntu to correctly run apt install
#  !apt install firefox-geckodriver
#  !cp /usr/lib/geckodriver /usr/bin
#  !cp /usr/lib/firefox /usr/bin
  !pip install selenium
  !apt-get update # to update ubuntu to correctly run apt install
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  import sys
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

initial_install()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
import os
import json
import random
import csv
import re

import multiprocessing
import logging
import threading
import time

# Selenium Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

# Import Firefox instance
from selenium.webdriver.firefox.options import Options

from IPython.display import clear_output

import pickle

import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Google Drive

In [3]:
#drive.mount('/content/drive')

## Definitions

In [4]:
def instantiate_driver():
    #options = Options()
    #options.add_argument('--headless')
    #options.add_argument('--ignore-certificate-errors')
    #options.add_argument('--incognito')
    #driver = webdriver.Firefox(options=options, executable_path='/usr/bin/geckodriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    return driver

def ByCheatSheet():
    by_values = [
            'ID = "id"',
            'XPATH = "xpath"',
            'LINK_TEXT = "link text"',
            'PARTIAL_LINK_TEXT = "partial link text"',
            'NAME = "name"',
            'TAG_NAME = "tag name"',
            'CLASS_NAME = "class name"',
            'CSS_SELECTOR = "css selector"',
    ]
    print(" \n".join(by_values))

def scraper(url): 
  with open(os.path.join(base_path,'error_log.txt'),'a') as file:
    # Log entry
    t0 = time.time()
    # Write log to file
    file.write(f"{url.split('-')[3].upper()} started \n") 
    # Instantiate driver instance
    driver_letter = instantiate_driver()
    # Call page (get request)
    driver_letter.get(url)
    # Find all elements in overview
    class_elements = driver_letter.find_elements(By.XPATH, '//*[@id="dictionary-top300-list__list-content_1-0"]/*')
    # Collect all elements with tag name 'a'
    title_hrefs = np.array([(element.text,element.get_attribute('href')) for element in class_elements if element.tag_name == 'a'])

    # Creation of intermediate df 
    df_ = pd.DataFrame(
      {
        'title':np.array([element[0] for element in title_hrefs]),
        'href':np.array([element[1] for element in title_hrefs]), 
        'explanation':np.zeros(np.array([element[0] for element in title_hrefs]).shape),
      }
    )

    for m,term in enumerate(title_hrefs[:,1]):
      # Xpath of introduction text
      xpath = '//*[@id="mntl-sc-page_1-0"]'# <- complete article | only introduction -> '//*[@id="mntl-sc-block_1-0-1"]'
      # Instantiate driver instance
      driver_term = instantiate_driver()
      # Call page (get request)
      driver_term.get(term)
      # Try and Except block for catching StaleElementReferenceException
      try:
        # Tries to find the respective text values. Waiting time is 10s
        explanation = WebDriverWait(driver_term, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).get_attribute('outerHTML')
      except:
        # Dummy value for explanation - NA values for df
        explanation = 'No explanation found'
        # Print value for console and log.txt file
        error = f'********** href: {term} did not work **********'
        print(error)
        file.write(f"{url.split('-')[3].upper()} | Element {term} not found \n") 

      # Assign value to intermediate df
      df_ = df_.assign(explanation = lambda x: np.where(x['href'] == term, explanation, x['explanation'])) 

      # Closes driver for term (driver inner loop)
      driver_term.quit()

    # Console output for orientation
    print(f"Letter: {url.split('-')[3]} - done")

    # Saves df as pickle file for post processing
    with open(os.path.join(base_path,f"{url.split('-')[3].upper()}_investopedia_glossary_Pickle"), 'ab') as f:
      pickle.dump(df_, f)
    
    # Log entry
    t1 = time.time()
    # Write log to file
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
    
    # Closes driver for letter (driver outer loop)
    driver_letter.quit()

## Scraping part
### Base Values

In [5]:
# Create dictonary of url basis
basedata = {
    'base' : 'https://www.investopedia.com/',
    'base_nr': 4769350,
    'direct': 'terms-beginning-with-{}-{}'}

# Create a list of letters in alphabet
alphabet = ['num','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
# Create iterator
keyword_url = [element for element in zip(alphabet, [num + basedata['base_nr'] for num in range(len(alphabet))])]
# create urls
urls = np.array([basedata['base'] + basedata['direct'].format(keyword.lower(), num) for keyword, num in keyword_url])

In [6]:
print(os.getcwd())
base_path = '/content/drive/MyDrive/NaturalLanguageUnderstanding/homework/data/ProTest'


/content


In [7]:
multiprocessing.cpu_count()

40

### Scraper

In [8]:
# Iterate through all possible urls for all letters (and number)
for n in range(len(urls)):
    exec(f't{n} = threading.Thread(target=scraper, args=[urls[n]])')
    exec(f't{n}.start()')
    
# Iterate through all possible urls for all letters (and number)
for n in range(len(urls)):
    exec(f't{n}.join()')

********** href: https://www.investopedia.com/business-essentials-4689832 did not work **********
Letter: x - done


Exception in thread Thread-35:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-e51bfa5db83e>", line 87, in scraper
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
TypeError: unsupported operand type(s) for -: 'float' and 'Thread'



********** href: https://www.investopedia.com/sectors-and-industries-analysis-4689756 did not work **********
********** href: https://www.investopedia.com/corporate-finance-and-accounting-4689821 did not work **********
********** href: https://www.investopedia.com/options-trading-strategy-and-education-4689661 did not work **********
********** href: https://www.investopedia.com/options-and-derivatives-trading-4689663 did not work **********
********** href: https://www.investopedia.com/career-advice-4689809 did not work **********
********** href: https://www.investopedia.com/budgeting-and-savings-4427755 did not work **********
********** href: https://www.investopedia.com/insurance-4427716 did not work **********
Letter: y - done


Exception in thread Thread-36:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-e51bfa5db83e>", line 87, in scraper
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
TypeError: unsupported operand type(s) for -: 'float' and 'Thread'



Letter: num - done


...

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-e51bfa5db83e>", line 87, in scraper
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
TypeError: unsupported operand type(s) for -: 'float' and 'Thread'



Letter: h - done


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-e51bfa5db83e>", line 87, in scraper
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
TypeError: unsupported operand type(s) for -: 'float' and 'Thread'



Letter: d - done


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-e51bfa5db83e>", line 87, in scraper
    file.write(f"{url.split('-')[3].upper()} ended at {t1-t2} \n")
TypeError: unsupported operand type(s) for -: 'float' and 'Thread'

